In this script I'll build the ANN for a classification

In [100]:
# Importing necessary libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
import sklearn.metrics as metrics 
import keras_tuner as kt
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import RandomUnderSampler
import time
import h5py

In [29]:
fetal_data = pd.read_csv("in/fetal_health.csv")
fetal_data.head()

,baseline value,accelerations,fetal_movement,uterine_contractions,light_decelerations,severe_decelerations,prolongued_decelerations,abnormal_short_term_variability,mean_value_of_short_term_variability,percentage_of_time_with_abnormal_long_term_variability,...,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance,histogram_tendency,fetal_health
0,132,0.006,0.000,0.006,0.003,0.0,0.0,17,2.1,0,...,68,198,6,1,141,136,140,12,0,1
1,133,0.003,0.000,0.008,0.003,0.0,0.0,16,2.1,0,...,68,198,5,1,141,135,138,13,0,1
2,134,0.003,0.000,0.008,0.003,0.0,0.0,16,2.4,0,...,53,170,11,0,137,134,137,13,1,1
3,132,0.007,0.000,0.008,0.000,0.0,0.0,16,2.4,0,...,53,170,9,0,137,136,138,11,1,1
4,131,0.005,0.072,0.008,0.003,0.0,0.0,28,1.4,0,...,88,154,5,0,135,134,137,7,1,1


In [30]:
fetal_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2597 entries, 0 to 2596
Data columns (total 22 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   baseline value                                          2597 non-null   int64  
 1   accelerations                                           2597 non-null   float64
 2   fetal_movement                                          2597 non-null   float64
 3   uterine_contractions                                    2597 non-null   float64
 4   light_decelerations                                     2597 non-null   float64
 5   severe_decelerations                                    2597 non-null   float64
 6   prolongued_decelerations                                2597 non-null   float64
 7   abnormal_short_term_variability                         2597 non-null   int64  
 8   mean_value_of_short_term_variability  

In [31]:
# Are the clases unbalanced? / Yes They are.
fetal_data.groupby("fetal_health").size()

fetal_health
1    1655
2     590
3     352
dtype: int64

Because of unbalanced data. The test/val size will be (5% for each one)

In [72]:
# Spliting data into features and labels
X = fetal_data.iloc[:,1:21]#.values
y = fetal_data["fetal_health"].astype(int)#.values

# Spliting the dataset into train, test and val
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05, random_state = 0)
abs_test_size = int(round(X.shape[0]*0.05,0))

# Split Train between train and validate / 268 is the 20% of the balanced dataset
X_val = X_train[:abs_test_size]
partial_X_train = X_train[abs_test_size:]

y_val = y_train[:abs_test_size]
partial_y_train = y_train[abs_test_size:]

# # Length
print('# Train:', partial_X_train.shape[0])
print('# Val:', X_val.shape[0])
print('# Test:', X_test.shape[0])

# Train: 2337
# Val: 130
# Test: 130


C:\Users\elvie\AppData\Local\Temp\ipykernel_10168\202329958.py:13: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_val = y_train[:abs_test_size]
C:\Users\elvie\AppData\Local\Temp\ipykernel_10168\202329958.py:14: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  partial_y_train = y_train[abs_test_size:]


In [75]:
# Are the train clases unbalanced? / Yes They are.
unbalanced_train = pd.DataFrame.copy(partial_X_train)
unbalanced_train["fetal_health"] = partial_y_train
unbalanced_train.groupby("fetal_health").size()

fetal_health
1    1499
2     517
3     321
dtype: int64

In [78]:
undersample = RandomUnderSampler(random_state=42)
X_train_bal, y_Train_bal = undersample.fit_resample(partial_X_train, partial_y_train)
print(y_Train_bal)

0      1
1      1
2      1
3      1
4      1
      ..
958    3
959    3
960    3
961    3
962    3
Name: fetal_health, Length: 963, dtype: int32


In [79]:
balanced_train = pd.DataFrame.copy(X_train_bal)
balanced_train["fetal_health"] = y_Train_bal.astype(int)
balanced_train.groupby("fetal_health").size()

fetal_health
1    321
2    321
3    321
dtype: int64

The amount of data is not much, so It's necessary to take this into account when the ANN classifier will be built.

Now, let's continue with the feature engineering part.

In [94]:
y_val[2490]

3

In [98]:
# Applying one-hot-econding method to the labels (y)
y_train_onehot = to_categorical(y_Train_bal)
y_val_onehot = to_categorical(y_val)
y_test_onehot = to_categorical(y_test)

# Print
print('Train labels example:', y_train_onehot[0], "is equal to:", y_Train_bal[0])
print('Val labels example:', y_val_onehot[0], "is equal to:", y_val[2490])
print('Test labels example:', y_test_onehot[0], "is equal to:", y_test[182])

Train labels example: [0. 1. 0. 0.] is equal to: 1
Val labels example: [0. 0. 0. 1.] is equal to: 3
Test labels example: [0. 1. 0. 0.] is equal to: 1


In [115]:
# PCA to features (X)
pca = PCA(n_components=4)  
pca.fit(X_train_bal)


PCA(n_components=4)

In [116]:
# Number of information that contribute each component.
pca.explained_variance_ratio_ 
print("Four of twenty components contribute to: ", round((sum(pca.explained_variance_ratio_ )*100),2),"%")

Four of twenty components contribute to:  94.03 %


In [121]:
# Apply the PCA to all X features
dt_train = pca.transform(X_train_bal)
dt_test = pca.transform(X_test)
dt_val = pca.transform(X_val)

In [125]:
dt_train.shape

(963, 4)

In [ ]:
# Building the ANN v 1.0
Model_1 = Sequential()
Model_1.add(Dense(64, activation='relu', input_shape=(4,)))
Model_1.add(Dense(128, activation='relu'))
Model_1.add(Dense(256, activation='relu'))
Model_1.add(Dense(512, activation='relu'))
Model_1.add(Dense(3, activation='softmax'))

Model_1.summary()

In [ ]:
Model_1.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
Hist_1 = Model_1.fit(dt_train,
                   y_train_onehot,
                   epochs=100,
                   batch_size=512,
                   validation_data=(dt_val, y_val_onehot))

In [ ]:
# Checking the overfitting
plt.plot(Hist_1.history['acc'], label='Train')
plt.plot(Hist_1.history['val_acc'], label='Val')
plt.legend()
plt.show()

In [ ]:
# Evaluate with test
Model_1.evaluate(dt_test, y_test_onehot)

In [ ]:
# Predict
prediction_1 = Model_1.predict(dt_test)
print(np.argmax(prediction_1[0]))
print(y_test_onehot[0])

Here I got the first Neural Network to predict the insurances charges. In this case there's a great error. It could be for the outlier that I found in the EDA part and because the target variables was not standardized  as well.

The idea is to build other models to compare what is the best one.